In [2]:
from fastapi import FastAPI, Query, HTTPException
from fastapi.middleware.cors import CORSMiddleware
import pandas as pd
import os
from typing import List, Dict, Optional

app = FastAPI(title="Excel Processor", docs_url="/docs")

app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

EXCEL_PATH = os.path.join("Data", "capbudg.xls")

def load_excel_sheets(path: str) -> Dict[str, pd.DataFrame]:
    try:
        return pd.read_excel(path, sheet_name=None, header=None)
    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))


@app.get("/list_tables")
def list_tables():
    try:
        sheets = load_excel_sheets(EXCEL_PATH)
        return {"tables": list(sheets.keys())}
    except Exception as e:
        raise HTTPException(status_code=500, detail="Unable to read Excel file.")


@app.get("/get_table_details")
def get_table_details(table_name: str = Query(...)):
    sheets = load_excel_sheets(EXCEL_PATH)
    if table_name not in sheets:
        raise HTTPException(status_code=404, detail="Table not found.")
    df = sheets[table_name]
    row_names = df.iloc[:, 0].dropna().astype(str).tolist()
    return {"table_name": table_name, "row_names": row_names}


@app.get("/row_sum")
def row_sum(table_name: str = Query(...), row_name: str = Query(...)):
    sheets = load_excel_sheets(EXCEL_PATH)
    if table_name not in sheets:
        raise HTTPException(status_code=404, detail="Table not found.")

    df = sheets[table_name]
    df = df.fillna(0)

    # Locate row
    match_row = df[df.iloc[:, 0].astype(str) == row_name]
    if match_row.empty:
        raise HTTPException(status_code=404, detail="Row name not found.")

    values = match_row.iloc[0, 1:]  # Skip first column (row name)
    numeric_values = pd.to_numeric(values.astype(str).str.replace('%', '', regex=False), errors='coerce').fillna(0)
    total = numeric_values.sum()

    return {"table_name": table_name, "row_name": row_name, "sum": total}
